In [0]:
import csv
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

### Test for CUDA

In [17]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
    

CUDA is available!  Training on GPU ...


### Load the Data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
'''
Load data from csv file
'''
def getBinaryfer13Data(filename):
    Y = []
    X = []
    first = True
    for line in open(filename, 'rU'):
        if first:
            first = False
        else:
            row = line.split(',')
            y = int(row[0])
            type = row[2]
            if y == 3 or y == 4:	#3=happy; 4=sad
                Y.append(abs(y-4))  #we want to store 1 for happy, 0 for sad
                X.append([int(p) for p in row[1].split()])
    return np.c_[np.array(X) / 255.0, np.array(Y)]

In [19]:
train_data = getBinaryfer13Data('fer3and4train.csv')
test_data = getBinaryfer13Data('fer3and4test.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: 'U' mode is deprecated
  


In [0]:
'''
Class for loading the custom dataset
Link: https://pytorch.org/docs/stable/data.html#map-style-datasets
'''
class imgData(object):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        temp = self.data[idx]
        x, y = temp[:-1], temp[-1]
        x = x.reshape(1,48,48).astype(float)
        return torch.tensor(x), torch.tensor(y, dtype=torch.long)


In [0]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20
# percentage of training data to use as validation
valid_size = 0.2

# choose the training and testing datasets
training_data = imgData(train_data)
testing_data = imgData(test_data)

# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(training_data, batch_size=batch_size,
                                      sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(training_data, batch_size=batch_size,
                                           sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(testing_data, batch_size=batch_size,
                                          num_workers=num_workers)

classes = ['happy', 'sad']

### Define the architecture

In [32]:
# define CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolutional layer (sees 48x48x1 image tensor)
        self.conv1 = nn.Conv2d(1, 4, 3, padding=1)
        # convolutional layer (sees 24x24x1 image tensor)
        self.conv2 = nn.Conv2d(4, 8, 3, padding=1)
        # max pooling layer
        self.pool = nn.MaxPool2d(2,2)
        # linear layer (8*12*12 -> 50)
        self.fc1 = nn.Linear(8*12*12, 50)
        # linear layer (50 -> 10)
        self.fc2 = nn.Linear(50, 2)
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.25)
        
    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # flatten image input
        x = x.view(-1, 8*12*12)
        # add dropout layer
        x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = self.fc2(x)
        return x

# create a complete CNN
model = Net()
model = model.double()
print(model)

# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()

Net(
  (conv1): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1152, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=2, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)


### Specify Loss function and Optimizer

In [0]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optmizer
optimizer = optim.SGD(model.parameters(), lr=0.01)

### Train the Network

In [34]:
# number of epochs to train the model
n_epochs = 20
# track change in validation loss
valid_loss_min = np.Inf
# keep track of traininf accuracy
correct = 0
# training accuracy
training_accuracy = 0

for epoch in range(1, n_epochs+1):

    # keep tracking of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    # keep track of training accuracy
    class_correct = list(0. for i in range(2))
    class_total = list(0. for i in range(2))
    
    ###################
    # train the model #
    ###################
    model.train()
    for batch_idx,(data, target) in enumerate(train_loader):
        print('epoch:', epoch, 'batch:', batch_idx)
        # move tensors to GPU if CUDA is available 
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target.long())
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)
        # compare predictions to true label
        correct_tensor = pred.eq(target.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
        # calculate training accuracy for each object class
        for i in range(batch_size):
            try:
                label = target.data[i]
                class_correct[label] += correct[i].item()
                class_total[label] += 1
            except:
                pass

    training_accuracy += np.sum(class_correct) / np.sum(class_total)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data,target in valid_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target.long())
        # update average validation loss
        valid_loss += loss.item()*data.size(0)
        
    # calculate average losses
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
    
    # print training/validation statistics
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_cifar.pt')
        valid_loss_min = valid_loss

epoch: 1 batch: 0
epoch: 1 batch: 1
epoch: 1 batch: 2
epoch: 1 batch: 3
epoch: 1 batch: 4
epoch: 1 batch: 5
epoch: 1 batch: 6
epoch: 1 batch: 7
epoch: 1 batch: 8
epoch: 1 batch: 9
epoch: 1 batch: 10
epoch: 1 batch: 11
epoch: 1 batch: 12
epoch: 1 batch: 13
epoch: 1 batch: 14
epoch: 1 batch: 15
epoch: 1 batch: 16
epoch: 1 batch: 17
epoch: 1 batch: 18
epoch: 1 batch: 19
epoch: 1 batch: 20
epoch: 1 batch: 21
epoch: 1 batch: 22
epoch: 1 batch: 23
epoch: 1 batch: 24
epoch: 1 batch: 25
epoch: 1 batch: 26
epoch: 1 batch: 27
epoch: 1 batch: 28
epoch: 1 batch: 29
epoch: 1 batch: 30
epoch: 1 batch: 31
epoch: 1 batch: 32
epoch: 1 batch: 33
epoch: 1 batch: 34
epoch: 1 batch: 35
epoch: 1 batch: 36
epoch: 1 batch: 37
epoch: 1 batch: 38
epoch: 1 batch: 39
epoch: 1 batch: 40
epoch: 1 batch: 41
epoch: 1 batch: 42
epoch: 1 batch: 43
epoch: 1 batch: 44
epoch: 1 batch: 45
epoch: 1 batch: 46
epoch: 1 batch: 47
epoch: 1 batch: 48
epoch: 1 batch: 49
epoch: 1 batch: 50
epoch: 1 batch: 51
epoch: 1 batch: 52
epo

### Training accuracy

In [35]:
print('Average Training Accuracy:', 100*training_accuracy/n_epochs, '%')

Average Training Accuracy: 63.55692530819435 %


### Load the Model with the Lowest Validation Loss

In [36]:
model.load_state_dict(torch.load('model_cifar.pt'))

<All keys matched successfully>

### Test the Trained Network

In [39]:
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))

model.eval()
# iterate over data
for data, target in test_loader:
    # mover tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the branch loss
    loss = criterion(output, target.long())
    # update test loss
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(batch_size):
        try:
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
        except:
            pass

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(2):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))


Test Loss: 0.535908

Test Accuracy of happy: 69% (856/1240)
Test Accuracy of   sad: 75% (1323/1760)

Test Accuracy (Overall): 72% (2179/3000)
